# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = 'https://raw.githubusercontent.com/Aakanksha743/Capstone_project/main/divorce.csv'
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config(path='Users/odl_user_246470/config.json')

# choose a name for experiment
experiment_name = 'capstoneproject1'

experiment=Experiment(ws, experiment_name)

In [4]:
compute_name = "optCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

optCompute exists already
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=trainCluster,
        training_data=ds,
        label_column_name='Class',
        n_cross_validations=5,
        **automl_settings
)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstoneproject1,AutoML_d0402c8a-9f16-4cf2-aede-86b3ccd68576,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstoneproject1,AutoML_d0402c8a-9f16-4cf2-aede-86b3ccd68576,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFea

{'runId': 'AutoML_d0402c8a-9f16-4cf2-aede-86b3ccd68576',
 'target': 'optCompute',
 'status': 'Completed',
 'startTimeUtc': '2023-12-23T11:51:44.690501Z',
 'endTimeUtc': '2023-12-23T12:04:58.775961Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'optCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"capstoneproject1","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-246470","workspace_name":"quick-starts-ws-246470","region":"westus2","compute_target":"optCompute","spark_ser

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: capstoneproject1,
Id: AutoML_d0402c8a-9f16-4cf2-aede-86b3ccd68576_8,
Type: None,
Status: Completed)
norm_macro_recall 0.9764705882352942
AUC_macro 1.0
recall_score_micro 0.9882352941176471
average_precision_score_macro 1.0
f1_score_macro 0.9881944444444445
recall_score_macro 0.9882352941176471
average_precision_score_weighted 1.0
f1_score_micro 0.9882352941176471
weighted_accuracy 0.9882352941176471
AUC_micro 0.9993079584775086
average_precision_score_micro 0.9993370681605975
f1_score_weighted 0.9881944444444445
balanced_accuracy 0.9882352941176471
matthews_correlation 0.9777046633277277
recall_score_weighted 0.9882352941176471
accuracy 0.9882352941176471
log_loss 0.06789772005940478
precision_score_micro 0.9882352941176471
precision_score_macro 0.9894736842105264
precision_score_weighted 0.9894736842105264
AUC_weighted 1.0
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_d0402c8a-9f16-4cf2-aede-86b3ccd68576_8/confusion_matrix
accuracy_table aml://artifactId/

In [9]:
#TODO: Save the best model
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook246470/code/Users/odl_user_246470'}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

ExtraTreesClassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 0.05,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.01,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 25,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': 

In [10]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-246470', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-246470'), name=capstoneModel_automl, id=capstoneModel_automl:2, version=2, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.9882352941176471'})


In [11]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

capstoneModel_automl version: 2
	 Training context : Auto ML
	 Accuracy : 0.9882352941176471


capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.9882352941176471




In [12]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "capstonedeploy", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-12-23 12:06:57+00:00 Creating Container Registry if not exists.
2023-12-23 12:06:57+00:00 Registering the environment.
2023-12-23 12:06:57+00:00 Use the existing image.
2023-12-23 12:06:58+00:00 Submitting deployment to compute.
2023-12-23 12:07:02+00:00 Checking the status of deployment capstonedeploy..
2023-12-23 12:09:05+00:00 Checking the status of inference endpoint capstonedeploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://bea5eeb8-425a-4c9d-a5b0-ef30ea974dfc.westus2.azurecontainer.io/score
http://bea5eeb8-425a-4c9d-a5b0-ef30ea974dfc.westus2.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
import json

#Import test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"Atr1": 4, "Atr2": 3, "Atr3": 4, "Atr4": 3, "Atr5": 4, "Atr6": 1, "Atr7": 0, "Atr8": 3, "Atr9": 4, "Atr10": 3, "Atr11": 4, "Atr12": 3, "Atr13": 4, "Atr14": 3, "Atr15": 4, "Atr16": 3, "Atr17": 4, "Atr18": 3, "Atr19": 4, "Atr20": 3, "Atr21": 4, "Atr22": 3, "Atr23": 4, "Atr24": 3, "Atr25": 4, "Atr26": 3, "Atr27": 4, "Atr28": 3, "Atr29": 4, "Atr30": 3, "Atr31": 4, "Atr32": 3, "Atr33": 4, "Atr34": 3, "Atr35": 4, "Atr36": 3, "Atr37": 4, "Atr38": 3, "Atr39": 4, "Atr40": 3, "Atr41": 4, "Atr42": 3, "Atr43": 4, "Atr44": 3, "Atr45": 4, "Atr46": 3, "Atr47": 4, "Atr48": 3, "Atr49": 4, "Atr50": 4, "Atr51": 3, "Atr52": 4, "Atr53": 3, "Atr54": 4}, {"Atr1": 3, "Atr2": 3, "Atr3": 2, "Atr4": 3, "Atr5": 3, "Atr6": 1, "Atr7": 1, "Atr8": 3, "Atr9": 3, "Atr10": 3, "Atr11": 4, "Atr12": 3, "Atr13": 3, "Atr14": 3, "Atr15": 3, "Atr16": 3, "Atr17": 3, "Atr18": 3, "Atr19": 4, "Atr20": 3, "Atr21": 3, "Atr22": 3, "Atr23": 3, "Atr24": 4, "Atr25": 3, "Atr26": 3, "Atr27": 2, "Atr28": 4, "Atr29": 3, "Atr30": 

In [17]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
# Print results from the inference
print(response.text)

"{\"result\": [1, 1, 0, 0, 1]}"


In [19]:
# Print original labels
print(label_df)

31     1
38     1
122    0
168    0
51     1
Name: Class, dtype: int64


In [20]:
print(service.get_logs())

2023-12-23T12:08:51,228654000+00:00 - rsyslog/run 
2023-12-23T12:08:51,240716100+00:00 - gunicorn/run 
2023-12-23T12:08:51,244940000+00:00 | gunicorn/run | 
2023-12-23T12:08:51,250570800+00:00 | gunicorn/run | ###############################################
2023-12-23T12:08:51,254308600+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-12-23T12:08:51,261433100+00:00 | gunicorn/run | ###############################################
2023-12-23T12:08:51,270516800+00:00 | gunicorn/run | 
2023-12-23T12:08:51,278053900+00:00 - nginx/run 
2023-12-23T12:08:51,287803600+00:00 | gunicorn/run | 
2023-12-23T12:08:51,303938000+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20231130.v5
2023-12-23T12:08:51,305679100+00:00 | gunicorn/run | 
2023-12-23T12:08:51,307878000+00:00 | gunicorn/run | 
2023-12-23T12:08:51,322597200+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
